This notebook is going to use Random Forest Algorithm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
trainRaw = pd.read_csv("../input/train_V2.csv")

In [ ]:
testRaw = pd.read_csv("../input/test_V2.csv")

In [ ]:
trainRaw[trainRaw['winPlacePerc'].isnull()]

Drop the NaN observation

In [ ]:
train = trainRaw.drop(2744604, axis=0)

![](http://)Headshot Rate might be an important feature. Idea brought from [this kernel](https://www.kaggle.com/rejasupotaro/cheaters-and-zombies).

In [ ]:
train.shape

In [ ]:
train['headshotRate'] = train['headshotKills'] / train['kills']
train['headshotRate'] = train['headshotRate'].fillna(0)

If there are more than one people in the group, then all people's rank will depend on the last died one. We find maximum and minimum values for 'assists', 'walkDistance', 'kills', 'DBNOs' columns then put them back to our data.

In [ ]:
groupMax = train[['groupId', 'assists', 'walkDistance', 'kills', 'DBNOs']].groupby('groupId').max()
groupMin = train[['groupId', 'assists', 'walkDistance', 'kills', 'DBNOs']].groupby('groupId').min()

In [ ]:
groupMax = groupMax.rename(columns = {'assists': 'assistsMax', 'walkDistance': 'walkDistanceMax', 'kills': 'killsMax', 'DBNOs': 'DBNOsMax'})
groupMin = groupMin.rename(columns = {'assists': 'assistsMin', 'walkDistance': 'walkDistanceMin', 'kills': 'killsMin', 'DBNOs': 'DBNOsMin'})

In [ ]:
groupMax.head()

In [ ]:
groupMin.head()

In [ ]:
dataAll = pd.merge(train, groupMax, on = 'groupId')
dataAll = pd.merge(dataAll, groupMin, on = 'groupId')

In [ ]:
dataAll.shape

In [ ]:
dataAll.head()

In [ ]:
dataAll.tail()

By the inspiration from killPlace, we think it might be helpful to get ranks in a match for more features.

In [ ]:
# matchRanks = train[['matchId', 'assists', 'boosts', 'damageDealt', 'heals', 'killStreaks', 'longestKill', 'walkDistance', \
#      'swimDistance', 'rideDistance']].groupby('matchId').rank(ascending = False)

In [ ]:
# matchRanks.head()

In [ ]:
# matchRanks = matchRanks.rename(columns = {'assists': 'assistsPlace', 'boosts': 'boostsPlace', 
#                                          'damageDealt': 'damageDealtPlace', 
#                                          'heals': 'heals', 'killStreaks': 'killStreaksPlace', 
#                                          'longestKill': 'longestKillPlace', 'walkDistance': 'walkDistancePlace', 
#                                          'swimDistance': 'swimDistancePlace', 'rideDistance': 'rideDistancePlace'})

In [ ]:
# matchRanks['matchId'] = train['matchId']

In [ ]:
# matchRanks.head()

In [ ]:
# matchRanks.tail()

In [ ]:
# matchRanks.shape

In [ ]:
# dataAll = pd.concat([dataAll.reset_index(drop=True),matchRanks.reset_index(drop=True)], axis=1)

In [ ]:
dataAll.shape

In [ ]:
dataAll.tail()

In [ ]:
dataAll.head()

In [ ]:
features = pd.get_dummies(dataAll.drop(['Id', 'groupId', 'matchId', 'winPlacePerc'], axis = 1))
labels = np.array(dataAll['winPlacePerc'])

In [ ]:
labels[np.isnan(labels)]

In [ ]:
features.head()

In [ ]:
print('features', features.shape, 'labels', labels.shape)

Train Model

TODO: Use cross-validation to play around and tweak parameters

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RFmodel = RandomForestRegressor(n_estimators=80, random_state=1937, n_jobs=3, min_samples_leaf=3, max_features='sqrt')

In [ ]:
print('start training')

In [ ]:
RFmodel.fit(features, labels)

Prepare test data

In [ ]:
test = testRaw.copy()

In [ ]:
test['headshotRate'] = test['headshotKills'] / test['kills']
test['headshotRate'] = test['headshotRate'].fillna(0)

In [ ]:
groupMaxTest = test[['groupId', 'assists', 'walkDistance', 'kills', 'DBNOs']].groupby('groupId').max()
groupMinTest = test[['groupId', 'assists', 'walkDistance', 'kills', 'DBNOs']].groupby('groupId').min()

In [ ]:
groupMaxTest = groupMaxTest.rename(columns = {'assists': 'assistsMax', 'walkDistance': 'walkDistanceMax', 'kills': 'killsMax', 'DBNOs': 'DBNOsMax'})
groupMinTest = groupMinTest.rename(columns = {'assists': 'assistsMin', 'walkDistance': 'walkDistanceMin', 'kills': 'killsMin', 'DBNOs': 'DBNOsMin'})

In [ ]:
test = pd.merge(test, groupMaxTest, on = 'groupId')

In [ ]:
test = pd.merge(test, groupMinTest, on = 'groupId')

In [ ]:
# matchRanksTest = test[['matchId', 'assists', 'boosts', 'damageDealt', 'heals', 'killStreaks', 'longestKill', 'walkDistance', \
#      'swimDistance', 'rideDistance']].groupby('matchId').rank()

In [ ]:
# matchRanksTest = matchRanksTest.rename(columns = {'assists': 'assistsPlace', 'boosts': 'boostsPlace', 
#                                           'damageDealt': 'damageDealtPlace', 
#                                           'heals': 'heals', 'killStreaks': 'killStreaksPlace', 
#                                           'longestKill': 'longestKillPlace', 'walkDistance': 'walkDistancePlace', 
#                                           'swimDistance': 'swimDistancePlace', 'rideDistance': 'rideDistancePlace'})

In [ ]:
# test = pd.concat([test.reset_index(drop=True),matchRanksTest.reset_index(drop=True)], axis=1)

In [ ]:
test.head()

In [ ]:
features.shape

In [ ]:
test = test.drop(['Id', 'groupId', 'matchId'],axis=1)

In [ ]:
test = pd.get_dummies(test)

In [ ]:
test.shape

In [ ]:
features.head()

In [ ]:
test.head()

Submission !!!

In [ ]:
prediction = RFmodel.predict(test)

In [ ]:
submission = pd.DataFrame({"Id":testRaw['Id'], "winPlacePerc":prediction})

In [ ]:
submission.head()

In [ ]:
submission.tail()

In [ ]:
submission.to_csv("submission.csv", index=False)